In [65]:
import pandas as pd, numpy as np, tensorflow as tf, gensim as gs, string, nltk, re, operator
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from matplotlib import pyplot
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adamnoack/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [66]:
data = pd.read_csv('data/spam.csv', sep=',', encoding='latin-1')
data = data.rename(index=str, columns={'v2':'x', 'v1':'y'})
data = data[['x', 'y']]
data.loc[data['y'] == 'ham', 'y'] = 0
data.loc[data['y'] == 'spam', 'y'] = 1
# data['x'] = data['x'].apply(lambda x: x.lower())
data['x'] = data['x'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print("Total samples: {}".format(data.index.size))
print("Spam {}".format(data[data['y'] == 1].index.size))
print("Ham {}".format(data[data['y'] == 0].index.size))

Total samples: 5572
Spam 747
Ham 4825


In [67]:
max_f = 2000
tokenizer = Tokenizer(num_words=max_f, split=' ')
tokenizer.fit_on_texts(data['x'].values)
X = tokenizer.texts_to_sequences(data['x'].values)
X = pad_sequences(X)

In [68]:
Y = pd.get_dummies(data['y']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, random_state = 42)

In [69]:
validation_size = int(data.index.size*.3*.5)
X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]

# sanity check
print("Training data shape: {}".format((X_train.shape,Y_train.shape)))
print("Validation data shape: {}".format((X_validate.shape,Y_validate.shape)))
print("Testing data shape: {}".format((X_test.shape,Y_test.shape)))

Training data shape: ((3900, 152), (3900, 2))
Validation data shape: ((835, 152), (835, 2))
Testing data shape: ((837, 152), (837, 2))


In [70]:
embed_dim = 128
lstm_out = 196

model = Sequential()
# instead of embedding the words in the data preprocessing stage, create an embedding layer
model.add(Embedding(max_f, embed_dim, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 152, 128)          256000    
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, 152, 128)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [71]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 1, validation_data=(X_validate, Y_validate), batch_size=batch_size, verbose = 1)

Train on 3900 samples, validate on 835 samples
Epoch 1/1
3900/3900 [==============================] - 47s 12ms/step - loss: 0.1840 - acc: 0.9367 - val_loss: 0.0882 - val_acc: 0.9796


In [72]:
score, acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.08
acc: 0.98
